In [ ]:
import numpy as np
import keras
from keras import backend as K
from keras.layers import Activation
from keras.layers.core import Dense, Flatten
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import *
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [ ]:
train_path="train"
valid_path='valid'
test_path='test'

In [ ]:
from keras.models import Sequential

In [ ]:
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D

In [ ]:
train_batches = ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True, vertical_flip=True, rescale=1/255., shear_range=0.1, rotation_range= 30).flow_from_directory(train_path, target_size=(224,224), classes=['Email','Memo','News','Report','Resume'], batch_size=32)
valid_batches = ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True, vertical_flip=True, rescale=1/255., shear_range=0.1, rotation_range= 20).flow_from_directory(valid_path, target_size=(224,224), classes=['Email','Memo','News','Report','Resume'], batch_size=32)
test_batches = ImageDataGenerator(rescale=1/255.).flow_from_directory(test_path, target_size=(224,224), classes=['Email','Memo','News','Report','Resume'], batch_size=10)

Found 100 images belonging to 5 classes.
Found 15 images belonging to 5 classes.
Found 10 images belonging to 5 classes.


In [ ]:
from keras.applications import VGG16
from keras import layers, models, backend as K

In [ ]:
K.clear_session()
base_model = VGG16(include_top=False, weights='imagenet',input_shape=(224,224,3))

In [ ]:
for l in base_model.layers:
  l.trainable = False

In [ ]:
x = layers.Flatten()(base_model.output)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.6)(x)
x = layers.Dense(200, activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.4)(x)
x = layers.Dense(100, activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(50, activation='relu')(x)
out = layers.Dense(5, activation='softmax')(x)
model_new = models.Model(base_model.input, out)
model_new.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [ ]:
model_new.compile(optimizer='rmsprop',loss='categorical_crossentropy', metrics=['acc'])
hist = model_new.fit_generator(train_batches, steps_per_epoch=5, validation_data=valid_batches, validation_steps=1, epochs=100, verbose=2)


Epoch 1/100
 - 68s - loss: 1.4723 - acc: 0.3750 - val_loss: 1.4626 - val_acc: 0.2667
Epoch 2/100
 - 88s - loss: 1.1366 - acc: 0.5188 - val_loss: 1.4793 - val_acc: 0.3333
Epoch 3/100
 - 59s - loss: 1.2578 - acc: 0.6250 - val_loss: 1.4093 - val_acc: 0.2667
Epoch 4/100
 - 76s - loss: 1.1007 - acc: 0.5758 - val_loss: 1.5120 - val_acc: 0.4000
Epoch 5/100
 - 69s - loss: 0.9022 - acc: 0.6288 - val_loss: 1.3084 - val_acc: 0.4000
Epoch 6/100
 - 55s - loss: 1.0961 - acc: 0.6250 - val_loss: 1.3297 - val_acc: 0.4667
Epoch 7/100
 - 74s - loss: 0.8143 - acc: 0.7121 - val_loss: 1.2242 - val_acc: 0.4000
Epoch 8/100
 - 73s - loss: 0.9285 - acc: 0.6061 - val_loss: 1.4672 - val_acc: 0.4000
Epoch 9/100
 - 68s - loss: 0.9072 - acc: 0.6515 - val_loss: 1.3366 - val_acc: 0.4667
Epoch 10/100
 - 68s - loss: 0.8951 - acc: 0.6970 - val_loss: 1.3990 - val_acc: 0.4000
Epoch 11/100
 - 59s - loss: 0.7810 - acc: 0.7212 - val_loss: 1.3438 - val_acc: 0.4000
Epoch 12/100
 - 73s - loss: 0.7825 - acc: 0.6894 - val_loss: 1

In [ ]:
import numpy as np
test_imgs, test_labels = next(test_batches)
print(test_labels)
y_test= [ np.where(r==1)[0][0] for r in test_labels ]
y_test

[[0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]]


[2, 3, 3, 4, 1, 2, 4, 1, 0, 0]

In [ ]:
p = model_new.predict(test_imgs)

In [ ]:
p.argmax(axis=1)

array([2, 3, 4, 4, 1, 2, 4, 1, 0, 0], dtype=int64)

In [ ]:
sum(p.argmax(axis=1) == y_test)/10*100

90.0